<a href="https://colab.research.google.com/github/aditanggoro11/RakaminKimiaFarma/blob/main/Project_DatabaseSainsData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Data

In [ ]:
import sqlite3
import pandas as pd

from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
conn = sqlite3.connect("/content/hotel.db")
cur = conn.cursor()

In [ ]:
cur.execute("SELECT * FROM Room LIMIT 5")
cur.fetchall()

[('R001', 1, 101, 'Deluxe', 1000000, 1),
 ('R002', 1, 102, 'Single', 600000, 1),
 ('R003', 1, 305, 'Double', 750000, 0),
 ('R004', 1, 404, 'Executive', 1400000, 1),
 ('R005', 1, 207, 'Suite', 1900000, 0)]

In [ ]:
# Fetch all table names
tables = cur.fetchall()

# Print the table names
print("Tables in the database:")
for table in tables:
    print(table[0])

Tables in the database:


In [ ]:
tables

[]

In [ ]:
# Utility: Generate next guest ID (e.g. G006)
def get_next_guest_id():
    cur.execute("SELECT guest_id FROM Guest ORDER BY guest_id DESC LIMIT 1")
    last = cur.fetchone()
    if last:
        last_id = int(last[0][1:])  # Extract number part after 'G'
        return f"G{last_id+1:03d}"
    else:
        return "G001"

# POV Manager

## Informasi Booking

In [ ]:
import sqlite3
import pandas as pd
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets

# --- Tampilkan Judul GUI ---
display(HTML("<h2>📋 Detail Pemesanan dan Informasi Terkait</h2>"))
# --- Koneksi ke database ---
conn = sqlite3.connect('hotel.db')
# --- Tombol Eksekusi Query ---
run_button_detail = widgets.Button(description="Tampilkan Detail Booking")
# --- Output untuk menampilkan hasil ---
output_detail = widgets.Output()
# --- Fungsi eksekusi saat tombol ditekan ---
def run_detail_query(b):
    with output_detail:
        clear_output()
        query = """
        SELECT
            b.booking_id,
            b.guest_id,
            g.name,
            g.email,
            g.phone,
            g.room_id,
            r.room_number,
            r.room_type,
            r.hotel_id,
            h.name AS hotel_name,
            h.city,
            h.rating,
            b.check_in_date,
            b.stay_duration_days,
            b.total_price
        FROM
            Booking b
        JOIN
            guest g ON b.guest_id = g.guest_id
        JOIN
            room r ON g.room_id = r.room_id
        JOIN
            hotel h ON r.hotel_id = h.hotel_id
        """
        try:
            df = pd.read_sql_query(query, conn)
            display(df)
        except Exception as e:
            print("Terjadi kesalahan:", e)
# Hubungkan tombol dengan fungsi
run_button_detail.on_click(run_detail_query)
# Tampilkan tombol dan output
display(widgets.VBox([run_button_detail, output_detail]))

## Add Guest

In [ ]:
import sqlite3
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Koneksi database
conn = sqlite3.connect('hotel.db')

# --- Judul ---
display(widgets.HTML(value="<h2>🧾 Tambah Data Tamu (Guest)</h2>"))

# --- Form input ---
name_input = widgets.Text(description="Name:")
email_input = widgets.Text(description="Email:")
phone_input = widgets.Text(description="Phone:")
room_id_input = widgets.IntText(description="Room ID:")
add_button = widgets.Button(description="Tambah Tamu", button_style='success')
add_output = widgets.Output()

# --- Fungsi tambah tamu ---
def add_guest(b):
    with add_output:
        clear_output()
        try:
            conn.execute(
                "INSERT INTO guest (name, email, phone, room_id) VALUES (?, ?, ?, ?)",
                (name_input.value, email_input.value, phone_input.value, room_id_input.value)
            )
            conn.commit()
            print(f"✅ Tamu '{name_input.value}' berhasil ditambahkan.")
        except Exception as e:
            print("❌ Terjadi kesalahan:", e)

add_button.on_click(add_guest)

# --- Tampilkan form ---
display(widgets.VBox([name_input, email_input, phone_input, room_id_input, add_button, add_output]))

HTML(value='<h2>🧾 Tambah Data Tamu (Guest)</h2>')

## Delete Guest

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Fungsi untuk mengambil ulang dropdown berdasarkan isi tabel guest terbaru
def get_guest_dropdown():
    df = pd.read_sql_query("SELECT guest_id, name FROM guest", conn)
    if df.empty:
        return widgets.Dropdown(options=[("Tidak ada tamu", None)], description="Pilih Tamu:")
    return widgets.Dropdown(
        options=[(row['name'], row['guest_id']) for _, row in df.iterrows()],
        description="Pilih Tamu:"
    )

# Fungsi untuk menjalankan proses hapus
def delete_guest(_):
    with output:
        clear_output()
        selected_id = guest_dropdown.value
        selected_name = guest_dropdown.label if hasattr(guest_dropdown, 'label') else guest_dropdown.label

        if selected_id is None:
            print("⚠️ Tidak ada tamu yang dipilih.")
            return

        cursor = conn.cursor()
        cursor.execute("DELETE FROM guest WHERE guest_id = ?", (selected_id,))
        conn.commit()
        print(f"🗑️ Tamu berhasil dihapus.")

        # Refresh dropdown setelah penghapusan
        refresh_delete_ui()

# Fungsi untuk menampilkan ulang UI hapus dengan dropdown terbaru
def refresh_delete_ui():
    global guest_dropdown
    guest_dropdown = get_guest_dropdown()
    delete_button = widgets.Button(description="Hapus Tamu", button_style="danger")
    delete_button.on_click(delete_guest)
    display(widgets.VBox([
        widgets.HTML(value="<h3>🗑️ Hapus Data Tamu</h3>"),
        guest_dropdown,
        delete_button,
        output
    ]))

output = widgets.Output()
refresh_delete_ui()

## Loyal Guest

In [ ]:
import sqlite3
import pandas as pd
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets

# --- Tampilkan Judul GUI ---
display(HTML("<h2>🌟 Daftar Loyal Guest</h2>"))

# --- Koneksi ke database ---
conn = sqlite3.connect('hotel.db')

# --- Tombol Eksekusi Query ---
run_button_loyal = widgets.Button(description="Tampilkan Loyal Guest")

# --- Output untuk menampilkan hasil ---
output_loyal = widgets.Output()

# --- Fungsi eksekusi saat tombol ditekan ---
def run_loyal_query(b):
    with output_loyal:
        clear_output()

        query = """
        SELECT
            g.name,
            g.email,
            g.phone,
            SUM(b.stay_duration_days) AS total_nights_stayed,
            COUNT(b.booking_id) AS total_bookings
        FROM
            guest g
        JOIN
            booking b ON g.guest_id = b.guest_id
        GROUP BY
            g.guest_id,
            g.name,
            g.email,
            g.phone
        HAVING
            SUM(b.stay_duration_days) >= 5
        ORDER BY
            total_nights_stayed DESC
        """

        try:
            df = pd.read_sql_query(query, conn)
            display(df)
        except Exception as e:
            print("Terjadi kesalahan:", e)

# Hubungkan tombol dengan fungsi
run_button_loyal.on_click(run_loyal_query)

# Tampilkan tombol dan output
display(widgets.VBox([run_button_loyal, output_loyal]))

## Dasbor Statistik Pemesanan

In [ ]:
import sqlite3
import pandas as pd
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets

# --- Tampilkan Judul GUI ---
display(HTML("<h2>🏨 Dasbor Statistik Pemesanan</h2>"))

# --- Koneksi ke database ---
conn = sqlite3.connect('hotel.db')

# --- Widget tanggal awal dan akhir ---
start_date = widgets.DatePicker(
    description='Start Date',
    value=pd.to_datetime('2025-05-01'),
    disabled=False
)

end_date = widgets.DatePicker(
    description='End Date',
    value=pd.to_datetime('2025-05-31'),
    disabled=False
)

# --- Tombol Eksekusi Query ---
run_button = widgets.Button(description="Tampilkan Statistik Booking")

# --- Output untuk menampilkan hasil ---
output = widgets.Output()

# --- Fungsi eksekusi saat tombol ditekan ---
def run_query(b):
    with output:
        clear_output()

        # Ambil tanggal dari widget
        start = start_date.value.strftime('%Y-%m-%d')
        end = end_date.value.strftime('%Y-%m-%d')

        # Query SQL
        query = f"""
        SELECT
            h.name AS hotel_name,
            h.city AS city,
            COUNT(b.booking_id) AS total_bookings,
            AVG(b.stay_duration_days) AS avg_stay_duration,
            SUM(b.stay_duration_days) AS total_nights_booked
        FROM
            hotel h
        JOIN
            room r ON r.hotel_id = h.hotel_id
        JOIN
            guest g ON r.room_id = g.room_id
        JOIN
            booking b ON g.guest_id = b.guest_id
        WHERE
            b.check_in_date BETWEEN '{start}' AND '{end}'
        GROUP BY
            hotel_name, city
        ORDER BY
            total_bookings DESC
        """

        try:
            df = pd.read_sql_query(query, conn)
            display(df)
        except Exception as e:
            print("Terjadi kesalahan:", e)

# Hubungkan tombol dengan fungsi
run_button.on_click(run_query)

# Tampilkan widget dan output
display(widgets.VBox([start_date, end_date, run_button, output]))

## Pendapatan Tipe Kamar dan Hotel

In [ ]:
import sqlite3
import pandas as pd
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets

# --- Tampilkan Judul GUI ---
display(HTML("<h2>💼 Statistik Pemesanan Berdasarkan Tipe Kamar</h2>"))

# --- Koneksi ke database ---
conn = sqlite3.connect('hotel.db')  # Ganti jika database-nya berbeda

# --- Ambil daftar hotel untuk dropdown ---
hotel_df = pd.read_sql_query("SELECT DISTINCT name FROM hotel", conn)
hotel_options = hotel_df['name'].tolist()
hotel_dropdown = widgets.Dropdown(
    options=hotel_options,
    description='Hotel:',
    value=hotel_options[0],
    disabled=False
)

# --- Widget tanggal awal dan akhir ---
start_date = widgets.DatePicker(
    description='Start Date',
    value=pd.to_datetime('2025-05-01'),
    disabled=False
)

end_date = widgets.DatePicker(
    description='End Date',
    value=pd.to_datetime('2025-05-31'),
    disabled=False
)

# --- Tombol eksekusi query ---
run_button = widgets.Button(description="Tampilkan Statistik")

# --- Output untuk hasil ---
output = widgets.Output()

# --- Fungsi eksekusi query ---
def run_query_with_hotel_filter(b):
    with output:
        clear_output()
        start = start_date.value.strftime('%Y-%m-%d')
        end = end_date.value.strftime('%Y-%m-%d')
        selected_hotel = hotel_dropdown.value

        query = f"""
        SELECT
            r.room_type,
            COUNT(b.booking_id) AS total_bookings,
            AVG(r.price_per_night) AS avg_price,
            SUM(b.stay_duration_days * r.price_per_night) AS estimated_revenue
        FROM
            booking b
        JOIN
            guest g ON b.guest_id = g.guest_id
        JOIN
            room r ON g.room_id = r.room_id
        JOIN
            hotel h ON r.hotel_id = h.hotel_id
        WHERE
            b.check_in_date BETWEEN '{start}' AND '{end}'
            AND h.name = '{selected_hotel}'
        GROUP BY
            r.room_type
        ORDER BY
            estimated_revenue DESC
        """

        try:
            df = pd.read_sql_query(query, conn)
            display(HTML(f"<b>Statistik untuk hotel:</b> {selected_hotel}"))
            display(df)
        except Exception as e:
            print("Terjadi kesalahan:", e)

# Hubungkan tombol ke fungsi
run_button.on_click(run_query_with_hotel_filter)

# Tampilkan GUI
display(widgets.VBox([
    hotel_dropdown,
    start_date,
    end_date,
    run_button,
    output
]))

# POV Guest

## Status dan Harga Kamar

In [ ]:
import sqlite3
import ipywidgets as widgets
from IPython.display import display, clear_output

# Koneksi ke database
conn = sqlite3.connect('/content/hotel.db')
cur = conn.cursor()

# Ambil daftar room_id dan room_type
cur.execute("SELECT room_id, room_type FROM Room")
rooms = cur.fetchall()

# Buat pilihan dropdown dari room_id dan room_type
room_dropdown = widgets.Dropdown(
    options=[(f"{room_id} ({room_type})", room_id) for room_id, room_type in rooms],
    description='Pilih Kamar:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# Tombol untuk submit
submit_button = widgets.Button(
    description='Lihat Harga',
    button_style='primary',
    tooltip='Klik untuk melihat harga kamar',
)

# Output widget
output = widgets.Output()

# Fungsi saat tombol ditekan
def on_button_clicked(b):
    with output:
        clear_output()
        room_id = room_dropdown.value
        cur.execute("SELECT price_per_night, is_available FROM Room WHERE room_id = ?", (room_id,))
        result = cur.fetchone()
        if result:
            price, available = result
            status = "Tersedia" if available else "Tidak Tersedia"
            print(f"Harga kamar per malam: Rp{price:,}")
            print(f"Status kamar: {status}")
        else:
            print("Kamar tidak ditemukan.")

submit_button.on_click(on_button_clicked)

# Tampilkan GUI
display(room_dropdown, submit_button, output)

Dropdown(description='Pilih Kamar:', layout=Layout(width='50%'), options=(('R001 (Deluxe)', 'R001'), ('R002 (S…

Button(button_style='primary', description='Lihat Harga', style=ButtonStyle(), tooltip='Klik untuk melihat har…

Output()

In [ ]:
import sqlite3
import ipywidgets as widgets
from IPython.display import display, clear_output

# Koneksi ke database
conn = sqlite3.connect('/content/hotel.db')
cur = conn.cursor()

# Ambil daftar hotel
cur.execute("SELECT hotel_id, name, rating FROM Hotel")
hotels = cur.fetchall()

# Dropdown hotel
hotel_dropdown = widgets.Dropdown(
    options=[(f"{name} - ⭐ {rating}", hotel_id) for hotel_id, name, rating in hotels],
    description='Pilih Hotel:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='60%')
)

# Dropdown kamar (kosong dulu, akan diisi setelah hotel dipilih)
room_dropdown = widgets.Dropdown(
    options=[],
    description='Pilih Kamar:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='60%')
)

# Tombol dan output
submit_button = widgets.Button(
    description='Lihat Harga',
    button_style='primary',
    tooltip='Klik untuk melihat harga kamar',
)
output = widgets.Output()

# Fungsi untuk update room_dropdown saat hotel dipilih
def on_hotel_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        hotel_id = change['new']
        cur.execute("SELECT room_id, room_type FROM Room WHERE hotel_id = ?", (hotel_id,))
        rooms = cur.fetchall()
        if rooms:
            room_dropdown.options = [(f"{room_id} ({room_type})", room_id) for room_id, room_type in rooms]
        else:
            room_dropdown.options = [("Tidak ada kamar", None)]

hotel_dropdown.observe(on_hotel_change)

# Fungsi tombol
def on_button_clicked(b):
    with output:
        clear_output()
        room_id = room_dropdown.value
        if room_id is None:
            print("⚠️ Pilih kamar yang tersedia terlebih dahulu.")
            return
        cur.execute("SELECT price_per_night, is_available FROM Room WHERE room_id = ?", (room_id,))
        result = cur.fetchone()
        if result:
            price, available = result
            status = "Tersedia" if available else "Tidak Tersedia"
            print(f"Harga kamar per malam: Rp{price:,}")
            print(f"Status kamar: {status}")
        else:
            print("Kamar tidak ditemukan.")

submit_button.on_click(on_button_clicked)

# Tampilkan GUI
display(widgets.HTML("<h3>🏨 Lihat Harga dan Status Kamar per Hotel</h3>"))
display(hotel_dropdown, room_dropdown, submit_button, output)

# Trigger awal untuk load rooms dari hotel pertama
hotel_dropdown.value = hotel_dropdown.value


HTML(value='<h3>🏨 Lihat Harga dan Status Kamar per Hotel</h3>')

Dropdown(description='Pilih Hotel:', layout=Layout(width='60%'), options=(('Pullman Jakarta Indonesia Thamrin …

Dropdown(description='Pilih Kamar:', layout=Layout(width='60%'), options=(), style=DescriptionStyle(descriptio…

Button(button_style='primary', description='Lihat Harga', style=ButtonStyle(), tooltip='Klik untuk melihat har…

Output()

## Find Your Perfect Room

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime, timedelta
import sqlite3

# Connect to database
conn = sqlite3.connect('hotel.db')
cur = conn.cursor()

## Utility Functions ##
def get_available_rooms(check_in=None, check_out=None, room_type=None, max_price=None):
    """Search for available rooms with filters"""
    query = """
        SELECT room_id, room_type, price_per_night, description
        FROM Room
        WHERE is_available = 1
    """
    params = []

    if room_type:
        query += " AND room_type = ?"
        params.append(room_type)

    if max_price:
        query += " AND price_per_night <= ?"
        params.append(max_price)

    if check_in and check_out:
        # Check for bookings that would conflict with these dates
        query += """
            AND room_id NOT IN (
                SELECT room_id FROM Booking
                JOIN Guest ON Booking.guest_id = Guest.guest_id
                WHERE ? < date(check_in_date, '+' || stay_duration_days || ' days')
                AND ? > check_in_date
            )
        """
        params.extend([check_in, check_out])

    cur.execute(query, params)
    return cur.fetchall()

## GUI Components ##

# Search Filters
check_in_date = widgets.DatePicker(
    description='Check-in:',
    value=datetime.today().date()
)

check_out_date = widgets.DatePicker(
    description='Check-out:',
    value=(datetime.today() + timedelta(days=1)).date()
)

room_type_dropdown = widgets.Dropdown(
    options=[('Any', None), ('Single', 'Single'), ('Double', 'Double'),
             ('Suite', 'Suite'), ('Deluxe', 'Deluxe')],
    description='Room Type:'
)

price_slider = widgets.IntSlider(
    value=500000,
    min=100000,
    max=2000000,
    step=50000,
    description='Max Price (Rp):',
    readout_format=',d'
)

search_button = widgets.Button(
    description='Search Rooms',
    button_style='primary',
    icon='search'
)

# Results Display
results_output = widgets.Output()

# Booking Form (hidden until room selected)
booking_form = widgets.VBox(visible=False)
selected_room = None

## Event Handlers ##

def on_search_button_click(b):
    global selected_room
    selected_room = None
    booking_form.visible = False

    with results_output:
        clear_output()
        check_in = check_in_date.value.strftime('%Y-%m-%d')
        check_out = check_out_date.value.strftime('%Y-%m-%d')

        if check_out <= check_in:
            print("⚠️ Error: Check-out date must be after check-in date")
            return

        rooms = get_available_rooms(
            check_in=check_in,
            check_out=check_out,
            room_type=room_type_dropdown.value,
            max_price=price_slider.value
        )

        if not rooms:
            print("No available rooms match your criteria.")
            return

        print(f"🏨 Found {len(rooms)} available rooms:")

        room_buttons = []
        for room in rooms:
            room_id, rtype, price, desc = room
            btn = widgets.Button(
                description=f"{room_id}: {rtype} - Rp{price:,}/night\n{desc}",
                layout=widgets.Layout(width='auto', height='auto')
            )

            def on_room_select(room=room):
                global selected_room
                selected_room = room
                show_booking_form()

            btn.on_click(on_room_select)
            room_buttons.append(btn)

        display(widgets.VBox(room_buttons))

def show_booking_form():
    global selected_room
    if not selected_room:
        return

    room_id, rtype, price, desc = selected_room
    check_in = check_in_date.value.strftime('%Y-%m-%d')
    check_out = check_out_date.value.strftime('%Y-%m-%d')
    nights = (check_out_date.value - check_in_date.value).days

    # Create form elements
    room_info = widgets.HTML(
        f"<h3>Booking {room_id}: {rtype}</h3>"
        f"<p>Rp{price:,} per night × {nights} nights = Rp{price*nights:,}</p>"
        f"<p>{desc}</p>"
    )

    name_input = widgets.Text(description="Full Name:")
    email_input = widgets.Text(description="Email:")
    phone_input = widgets.Text(description="Phone:")
    special_requests = widgets.Textarea(description="Special Requests:")

    book_button = widgets.Button(
        description="Confirm Booking",
        button_style='success'
    )

    def on_book_button_click(b):
        try:
            # Generate IDs
            guest_id = f"G{datetime.now().strftime('%Y%m%d%H%M%S')}"
            booking_id = f"B{datetime.now().strftime('%Y%m%d%H%M%S')}"

            # Insert guest
            cur.execute(
                "INSERT INTO Guest (guest_id, name, email, phone) VALUES (?, ?, ?, ?)",
                (guest_id, name_input.value, email_input.value, phone_input.value)
            )

            # Insert booking
            cur.execute(
                """INSERT INTO Booking
                (booking_id, guest_id, room_id, check_in_date, stay_duration_days, total_price)
                VALUES (?, ?, ?, ?, ?, ?)""",
                (booking_id, guest_id, room_id, check_in, nights, price*nights)
            )

            # Update room availability
            cur.execute(
                "UPDATE Room SET is_available = 0 WHERE room_id = ?",
                (room_id,)
            )

            conn.commit()

            with results_output:
                clear_output()
                print(f"✅ Booking confirmed! Your booking ID is: {booking_id}")
                print(f"We've sent details to {email_input.value}")

            booking_form.visible = False

        except Exception as e:
            with results_output:
                print(f"❌ Error: {str(e)}")
            conn.rollback()

    book_button.on_click(on_book_button)

    # Update booking form
    booking_form.children = [
        room_info,
        name_input,
        email_input,
        phone_input,
        special_requests,
        book_button
    ]
    booking_form.visible = True

## Main Layout ##

search_button.on_click(on_search_button_click)

search_panel = widgets.VBox([
    widgets.HTML("<h2>🔍 Find Your Perfect Room</h2>"),
    check_in_date,
    check_out_date,
    room_type_dropdown,
    price_slider,
    search_button
])

main_layout = widgets.VBox([
    search_panel,
    results_output,
    booking_form
])

display(main_layout)

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime, timedelta
import sqlite3

# Connect to database
conn = sqlite3.connect('hotel.db')
cur = conn.cursor()

## Utility Functions ##

def get_hotels():
    """Get all hotels from database"""
    cur.execute("SELECT hotel_id, name, city, rating FROM Hotel")
    return cur.fetchall()

def get_available_rooms(hotel_id, check_in=None, check_out=None, room_type=None, max_price=None):
    """Search for available rooms with filters"""
    query = """
        SELECT room_id, room_number, room_type, price_per_night
        FROM Room
        WHERE hotel_id = ? AND is_available = 1
    """
    params = [hotel_id]

    if room_type:
        query += " AND room_type = ?"
        params.append(room_type)

    if max_price:
        query += " AND price_per_night <= ?"
        params.append(max_price)

    if check_in and check_out:
        # Check for bookings that would conflict with these dates
        query += """
            AND room_id NOT IN (
                SELECT room_id FROM Booking
                JOIN Guest ON Booking.guest_id = Guest.guest_id
                WHERE ? < date(check_in_date, '+' || stay_duration_days || ' days')
                AND ? > check_in_date
            )
        """
        params.extend([check_in, check_out])

    cur.execute(query, params)
    return cur.fetchall()

## GUI Components ##

# Hotel Selection
hotel_dropdown = widgets.Dropdown(
    options=[(f"{h[1]} ({h[2]}) ⭐{h[3]}", h[0]) for h in get_hotels()],
    description='Select Hotel:'
)

# Search Filters
check_in_date = widgets.DatePicker(
    description='Check-in:',
    value=datetime.today().date()
)

check_out_date = widgets.DatePicker(
    description='Check-out:',
    value=(datetime.today() + timedelta(days=1)).date()
)

room_type_dropdown = widgets.Dropdown(
    options=[('Any', None), ('Single', 'Single'), ('Double', 'Double'),
             ('Suite', 'Suite'), ('Deluxe', 'Deluxe')],
    description='Room Type:'
)

price_slider = widgets.IntSlider(
    value=500000,
    min=100000,
    max=2000000,
    step=50000,
    description='Max Price (Rp):',
    readout_format=',d'
)

search_button = widgets.Button(
    description='Search Rooms',
    button_style='primary',
    icon='search'
)

# Results Display
results_output = widgets.Output()

# Booking Form (hidden until room selected)
booking_form = widgets.VBox(visible=False)
selected_room = None

## Event Handlers ##

def on_search_button_click(b):
    global selected_room
    selected_room = None
    booking_form.visible = False

    with results_output:
        clear_output()
        check_in = check_in_date.value.strftime('%Y-%m-%d')
        check_out = check_out_date.value.strftime('%Y-%m-%d')

        if check_out <= check_in:
            print("⚠️ Error: Check-out date must be after check-in date")
            return

        hotel_id = hotel_dropdown.value
        rooms = get_available_rooms(
            hotel_id=hotel_id,
            check_in=check_in,
            check_out=check_out,
            room_type=room_type_dropdown.value,
            max_price=price_slider.value
        )

        if not rooms:
            print("No available rooms match your criteria.")
            return

        print(f"🏨 Found {len(rooms)} available rooms:")

        room_buttons = []
        for room in rooms:
            room_id, room_number, rtype, price = room
            btn = widgets.Button(
                description=f"Room #{room_number}: {rtype} - Rp{price:,}/night",
                layout=widgets.Layout(width='auto', height='auto')
            )

            def on_room_select(room=room):
                global selected_room
                selected_room = room
                show_booking_form()

            btn.on_click(on_room_select)
            room_buttons.append(btn)

        display(widgets.VBox(room_buttons))

def show_booking_form():
    global selected_room
    if not selected_room:
        return

    room_id, room_number, rtype, price = selected_room
    check_in = check_in_date.value.strftime('%Y-%m-%d')
    check_out = check_out_date.value.strftime('%Y-%m-%d')
    nights = (check_out_date.value - check_in_date.value).days

    # Create form elements
    room_info = widgets.HTML(
        f"<h3>Booking Room #{room_number}: {rtype}</h3>"
        f"<p>Rp{price:,} per night × {nights} nights = Rp{price*nights:,}</p>"
    )

    name_input = widgets.Text(description="Full Name:")
    email_input = widgets.Text(description="Email:")
    phone_input = widgets.Text(description="Phone:")
    special_requests = widgets.Textarea(description="Special Requests:")

    book_button = widgets.Button(
        description="Confirm Booking",
        button_style='success'
    )

    def on_book_button_click(b):
        try:
            # Generate IDs
            guest_id = f"G{datetime.now().strftime('%Y%m%d%H%M%S')}"
            booking_id = f"B{datetime.now().strftime('%Y%m%d%H%M%S')}"

            # Insert guest
            cur.execute(
                "INSERT INTO Guest (guest_id, name, email, phone) VALUES (?, ?, ?, ?)",
                (guest_id, name_input.value, email_input.value, phone_input.value)
            )

            # Insert booking
            cur.execute(
                """INSERT INTO Booking
                (booking_id, guest_id, room_id, check_in_date, stay_duration_days, total_price)
                VALUES (?, ?, ?, ?, ?, ?)""",
                (booking_id, guest_id, room_id, check_in, nights, price*nights)
            )

            # Update room availability
            cur.execute(
                "UPDATE Room SET is_available = 0 WHERE room_id = ?",
                (room_id,)
            )

            conn.commit()

            with results_output:
                clear_output()
                print(f"✅ Booking confirmed! Your booking ID is: {booking_id}")
                print(f"We've sent details to {email_input.value}")

            booking_form.visible = False

        except Exception as e:
            with results_output:
                print(f"❌ Error: {str(e)}")
            conn.rollback()

    book_button.on_click(on_book_button)

    # Update booking form
    booking_form.children = [
        room_info,
        name_input,
        email_input,
        phone_input,
        special_requests,
        book_button
    ]
    booking_form.visible = True

## Main Layout ##

search_button.on_click(on_search_button_click)

search_panel = widgets.VBox([
    widgets.HTML("<h2>🔍 Find Your Perfect Room</h2>"),
    hotel_dropdown,
    check_in_date,
    check_out_date,
    room_type_dropdown,
    price_slider,
    search_button
])

main_layout = widgets.VBox([
    search_panel,
    results_output,
    booking_form
])

display(main_layout)

## Submit Booking

In [ ]:
# Utility: Generate next booking ID (e.g. B006)
def get_next_booking_id():
    cur.execute("SELECT booking_id FROM Booking ORDER BY booking_id DESC LIMIT 1")
    last = cur.fetchone()
    if last:
        last_id = int(last[0][1:])
        return f"B{last_id+1:03d}"
    else:
        return "B001"

# View all bookings
def show_bookings(_=None):
    clear_output()
    display(main_menu)
    cur.execute("""
        SELECT B.booking_id, G.name, B.check_in_date, B.stay_duration_days, B.total_price
        FROM Booking B
        JOIN Guest G ON B.guest_id = G.guest_id
    """)
    results = cur.fetchall()
    print("📋 All Bookings:")
    for row in results:
        print(f"{row[0]} | Guest: {row[1]} | Check-in: {row[2]} | Duration: {row[3]} | Price: Rp{row[4]:,.0f}")

def get_available_rooms():
    cur.execute("SELECT room_id FROM Room WHERE is_available = 1")
    return [r[0] for r in cur.fetchall()]

def add_booking_form(_=None):
    clear_output()
    display(main_menu)

    rooms = get_available_rooms()
    if not rooms:
        print("⚠️ No rooms available.")
        return

    # Guest info
    name_input = widgets.Text(description="👤 Name:")
    email_input = widgets.Text(description="📧 Email:")
    phone_input = widgets.Text(description="📞 Phone:")

    # Booking info
    room_dropdown = widgets.Dropdown(options=rooms, description="🛏️ Room:")
    checkin_input = widgets.Text(value=str(datetime.today().date()), description="📅 Check-in:")
    duration_input = widgets.IntText(value=1, description="🕒 Days:")
    price_input = widgets.IntText(value=0, description="💵 Price:", disabled=True)  # Auto-filled

    submit_btn = widgets.Button(description="Submit Booking", button_style='success')

    # Update price when room is selected
    def update_price(change):
        room_id = change['new']
        cur.execute("SELECT price_per_night FROM Room WHERE room_id = ?", (room_id,))
        result = cur.fetchone()
        if result:
            price_input.value = result[0] * duration_input.value
        else:
            price_input.value = 0

    # Update price when duration is changed
    def update_total_price(change):
        room_id = room_dropdown.value
        cur.execute("SELECT price_per_night FROM Room WHERE room_id = ?", (room_id,))
        result = cur.fetchone()
        if result:
            price_input.value = result[0] * change['new']

    room_dropdown.observe(update_price, names='value')
    duration_input.observe(update_total_price, names='value')

    # Trigger initial price update
    update_price({'new': room_dropdown.value})

    def submit(b):
        guest_id = get_next_guest_id()
        booking_id = get_next_booking_id()
        name = name_input.value
        email = email_input.value
        phone = phone_input.value
        room_id = room_dropdown.value
        checkin = checkin_input.value
        duration = duration_input.value
        price = price_input.value

        try:
            # Insert into Guest
            cur.execute("""
                INSERT INTO Guest (guest_id, name, email, phone, room_id)
                VALUES (?, ?, ?, ?, ?)
            """, (guest_id, name, email, phone, room_id))

            # Insert into Booking
            cur.execute("""
                INSERT INTO Booking (booking_id, guest_id, check_in_date, stay_duration_days, total_price)
                VALUES (?, ?, ?, ?, ?)
            """, (booking_id, guest_id, checkin, duration, price))

            # Update room availability
            cur.execute("UPDATE Room SET is_available = 0 WHERE room_id = ?", (room_id,))
            conn.commit()
            print(f"✅ Booking '{booking_id}' for guest '{name}' added.")
            show_bookings()
        except Exception as e:
            print(f"❌ Error: {e}")
            conn.rollback()

    submit_btn.on_click(submit)

    display(widgets.VBox([
        widgets.HTML("<h3>🧾 New Guest & Booking</h3>"),
        name_input, email_input, phone_input,
        room_dropdown, checkin_input, duration_input, price_input,
        submit_btn
    ]))

# Buttons
show_btn = widgets.Button(description="📋 Show All Bookings", button_style='info')
add_btn = widgets.Button(description="➕ Add Booking", button_style='primary')

show_btn.on_click(show_bookings)
add_btn.on_click(add_booking_form)

# Main menu
main_menu = widgets.HBox([show_btn, add_btn])
display(main_menu)

In [ ]:
import sqlite3
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output

# Connect
conn = sqlite3.connect("/content/hotel.db")
cur = conn.cursor()

# Helpers
def get_next_booking_id():
    cur.execute("SELECT booking_id FROM Booking ORDER BY booking_id DESC LIMIT 1")
    last = cur.fetchone()
    return f"B{int(last[0][1:]) + 1:03}" if last else "B001"

def get_next_guest_id():
    cur.execute("SELECT guest_id FROM Guest ORDER BY guest_id DESC LIMIT 1")
    last = cur.fetchone()
    return f"G{int(last[0][1:]) + 1:03}" if last else "G001"

def get_available_rooms():
    cur.execute("SELECT room_id FROM Room WHERE is_available = 1")
    return [r[0] for r in cur.fetchall()]

# View bookings
def show_bookings(_=None):
    clear_output()
    display(main_menu)
    cur.execute("""
        SELECT B.booking_id, G.name, B.check_in_date, B.stay_duration_days, B.total_price
        FROM Booking B
        JOIN Guest G ON B.guest_id = G.guest_id
    """)
    results = cur.fetchall()
    print("📋 Bookings:")
    for row in results:
        print(f"{row[0]} | {row[1]} | Check-in: {row[2]} | Days: {row[3]} | Price: Rp{row[4]:,.0f}")

# Add new guest and booking
def add_booking_form(_=None):
    clear_output()
    display(main_menu)

    rooms = get_available_rooms()
    if not rooms:
        print("⚠️ No rooms available.")
        return

    # Guest info
    name_input = widgets.Text(description="👤 Name:")
    email_input = widgets.Text(description="📧 Email:")
    phone_input = widgets.Text(description="📞 Phone:")

    # Booking info
    room_dropdown = widgets.Dropdown(options=rooms, description="🛏️ Room:")
    checkin_input = widgets.Text(value=str(datetime.today().date()), description="📅 Check-in:")
    duration_input = widgets.IntText(value=1, description="🕒 Days:")
    price_input = widgets.IntText(value=1000000, description="💵 Price:")
    submit_btn = widgets.Button(description="Submit Booking", button_style='success')

    def submit(b):
        guest_id = get_next_guest_id()
        booking_id = get_next_booking_id()
        name = name_input.value
        email = email_input.value
        phone = phone_input.value
        room_id = room_dropdown.value
        checkin = checkin_input.value
        duration = duration_input.value
        price = price_input.value

        try:
            # Insert into Guest
            cur.execute("""
                INSERT INTO Guest (guest_id, name, email, phone, room_id)
                VALUES (?, ?, ?, ?, ?)
            """, (guest_id, name, email, phone, room_id))

            # Insert into Booking
            cur.execute("""
                INSERT INTO Booking (booking_id, guest_id, check_in_date, stay_duration_days, total_price)
                VALUES (?, ?, ?, ?, ?)
            """, (booking_id, guest_id, checkin, duration, price))

            # Update room availability
            cur.execute("UPDATE Room SET is_available = 0 WHERE room_id = ?", (room_id,))
            conn.commit()
            print(f"✅ Booking '{booking_id}' for guest '{name}' added.")
            show_bookings()
        except Exception as e:
            print(f"❌ Error: {e}")
            conn.rollback()

    submit_btn.on_click(submit)

    display(widgets.VBox([
        widgets.HTML("<h3>🧾 New Guest & Booking</h3>"),
        name_input, email_input, phone_input,
        room_dropdown, checkin_input, duration_input, price_input,
        submit_btn
    ]))

# Main Menu
show_btn = widgets.Button(description="📋 Show Bookings", button_style='info')
add_btn = widgets.Button(description="➕ New Guest & Booking", button_style='primary')
show_btn.on_click(show_bookings)
add_btn.on_click(add_booking_form)
main_menu = widgets.HBox([show_btn, add_btn])

display(main_menu)
